In [1]:
import torch
import torchvision
import re


train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True)
test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=torchvision.transforms.ToTensor())

In [5]:
all_labels = set(test_dataset.targets.tolist())
dataset_splited = dict()
for label in all_labels:
    print(type(label), type(test_dataset.targets.tolist()))
    list_index = [i.start() for i in re.finditer(label, test_dataset.targets.tolist())]

<class 'int'> <class 'list'>


TypeError: first argument must be string or compiled pattern